# 5. Imbalanced classes

## Synthesize and Modeling the Data 

Oftenly in credit scoring use-cases and mainly in porfolios of secured loans as mortage loans, there is a significant presence of class imbalancement. Class imbalance occurs when the number of a certain class outweights the number of other classes present in the dataset. If a naive classifier model is used under the presence of imbalanced classes, it can achieve high accuracy by assigning all cases to the majority class - but this is very damaging for a a business and can have high costs associated with it.



Since we had earlier diagnosed an imbalanced class problem, using YData's state-of-the-art synthesizers, we can generate synthetic data for the minority class and balance it.

However, we would only do so for the training dataset, thus continuously evaluating the performance against the real unaltered test dataset.

### Import the needed packages

In [1]:
import os
import sys

import pickle 

import pandas as pd

from balance_model_training import train_model, augment_minority

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
try:
    augment=int(os.getenv('AUGMENT'))
except: 
    augment=0

In [4]:
prep = pickle.load(open('prep_parameters.pkl', 'rb'))

### Get the training dataset

In [5]:
data = pd.read_csv('prep_traindata.csv', index_col=[0])

In [6]:
if augment==1:
    result, models = train_model(X=data, label='SeriousDlqin2yrs', augmentation=True, train_synth=True)
else:
    result, models = train_model(X=data, label='SeriousDlqin2yrs', augmentation=False, train_synth=False)
    
prep['Balancing'] = augment

90950
(7025, 10)


[                                        ] | 0% Completed | 242.90 us

[##                                      ] | 5% Completed | 107.97 ms

[####                                    ] | 10% Completed | 213.74 ms

[##########                              ] | 26% Completed | 318.93 ms

...

INFO: 2022-10-20 22:15:16,574 [SYNTHESIZER] - Processing the data prior fitting the synthesizer.


INFO: 2022-10-20 22:15:18,746 [SYNTHESIZER] - Start generating model samples.


Size training: 195950
Size test: 45000
Model training: DummyClassifier
Model training: RandomForestClassifier


Model training: AdaBoostClassifier


Model training: DecisionTreeClassifier


In [7]:
#Save the best model? Does it make sense?
#Here save the models dict? What can I do about this
optimized_model = models[result.set_index('model')['f1_score'].idxmax()]

In [8]:
#Here the model needs also to be an output to be shown
pickle.dump(optimized_model, open('best_model.pkl', 'wb'))
pickle.dump(prep, open('prep_parameters.pkl', 'wb'))

### Creating the pipeline step outputs

In [9]:
import json 

metadata = {
    'outputs' : [
        {
          'type': 'markdown',
          'storage': 'inline',
          'source': f'## **Dataset balancing:** {bool(augment)}',
        },
        {
          'type': 'table',
          'storage': 'inline',
          'format': 'csv',
          'header': list(result.columns),
          'source': result.to_csv(header=False, index=False)
        },
    ]
  }

#heatmap_output(data=data_mask.corr()[mask_cols].drop(missing_col+mask_cols), title='Missing correlation')
with open('mlpipeline-ui-metadata.json', 'w') as metadata_file:
    json.dump(metadata, metadata_file)